In [ ]:
import requests
import pandas as pd

def get_rodada_df(req):
    partida_content = []
    for partida in req.json():
        equipe_mandante = partida['equipes']['mandante']['sigla']
        equipe_visitante = partida['equipes']['visitante']['sigla']
        gols_mandante = partida['placar_oficial_mandante']
        gols_visitante = partida['placar_oficial_visitante']
        sg_mandante = gols_mandante - gols_visitante
        sg_visitante = gols_visitante - gols_mandante
        if gols_mandante > gols_visitante:
            pts_mandante = 3
            pts_visitante = 0
        elif gols_visitante > gols_mandante:
            pts_mandante = 0
            pts_visitante = 3
        elif gols_mandante == gols_visitante:
            pts_mandante = 1
            pts_visitante = 1
        partida_content.append([equipe_mandante, pts_mandante, gols_mandante, gols_visitante, sg_mandante])
        partida_content.append([equipe_visitante, pts_visitante, gols_visitante, gols_mandante, sg_visitante])
    
    return pd.DataFrame(partida_content, columns=['EQUIPE', 'PONTOS', 'GP', 'GC', 'SG'])


url = r"https://api.globoesporte.globo.com/tabela/009b5a68-dd09-46b8-95b3-293a2d494366/fase/brasileiro-serie-b-2024-fase-unica/rodada/{}/jogos/"

complete_championship = pd.DataFrame()
for match_number in range(1, 37):
    req_rodada = requests.get(url.format(match_number))
    df_rodada = get_rodada_df(req=req_rodada)
    if match_number == 1:
        df_rodada['RODADA'] = match_number
        complete_championship = pd.concat([complete_championship, df_rodada])
    else:
        copy_acumulated = complete_championship.copy()
        copy_acumulated = copy_acumulated.query('RODADA == @match_number-1').drop(columns=['RODADA'])
        new_df_rodada = pd.concat([copy_acumulated, df_rodada],ignore_index=True).groupby(['EQUIPE']).sum().reset_index()
        new_df_rodada['RODADA'] = match_number
        complete_championship = pd.concat([complete_championship, new_df_rodada])
complete_championship.to_excel('classificacao_serie_b.xlsx', index=False)